In [1]:
import docker
d = docker.DockerClient()


In [2]:
import socketio

In [4]:
import asyncio
import socketio

sio = socketio.AsyncClient()




@sio.on('*')
def catch_all(event, data):
    print(data)



await sio.connect('http://localhost:5000/')
await sio.wait()



ConnectionError: Connection refused by the server